# 03. Feature Extraction

In [14]:
# 01. CONFIGURAÇÃO E EPOCAGEM

import os
import mne
import warnings
import numpy as np
import pandas as pd

mne.set_log_level('WARNING')
warnings.filterwarnings('ignore')

# 1. Diretórios
DIR_CLEANED = '../data/interim/cleaned_data/'
DIR_EPOCHS = '../data/processed/epochs/'
DIR_REPORTS = '../reports/'

for d in [DIR_EPOCHS, DIR_REPORTS]:
    os.makedirs(d, exist_ok=True)

# 2. Dicionário de Marcações
EVENT_DICT = {
    'FF': 1, 'F': 1,  # Face Feliz
    'FN': 2, 'N': 2,  # Face Neutra
    'FR': 3, 'R': 3   # Face Raiva
}

# 3. Parâmetros das Frequências
BANDAS_FREQ = {
    'Theta': (4.0, 8.0),
    'Alpha': (8.0, 13.0),
    'Beta': (13.0, 30.0),
    'Gamma': (30.0, 40.0)
}

CANAIS_PSD = ['F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2']
CANAIS_N170 = ['O1', 'O2']
JANELA_N170 = (0.130, 0.230)

# Segmentação em lote (epocagem)
arquivos_limpos = sorted([f for f in os.listdir(DIR_CLEANED) if f.endswith('.fif')])
print(f"Iniciando epocagem para {len(arquivos_limpos)} blocos...")

for arquivo in arquivos_limpos:
    paciente_id = arquivo.replace('-cleaned-raw.fif', '')
    caminho_entrada = os.path.join(DIR_CLEANED, arquivo)
    caminho_saida = os.path.join(DIR_EPOCHS, f"{paciente_id}-epo.fif")
          
    try:
        raw = mne.io.read_raw_fif(caminho_entrada, preload=True, verbose=False)
        
        # Lógica para o bloco de FACES
        if 'faces' in arquivo:
            # O Regex aceita FF, FN, FR OU F, N, R isolados
            events, event_id_encontrados = mne.events_from_annotations(
                raw, 
                event_id=EVENT_DICT, 
                regexp='^(FF|FN|FR|F|N|R)$', 
                verbose=False
            )
            
            if len(events) > 0:
                epochs = mne.Epochs(raw, events, event_id=event_id_encontrados, 
                                    tmin=-0.2, tmax=0.8, baseline=(None, 0), preload=True, verbose=False)
                epochs.save(caminho_saida, overwrite=True, verbose=False)
                print(f" {paciente_id}: Sucesso ({len(epochs)} épocas)")
            else:
                print(f" Atenção {paciente_id}: Nenhuma marcação de face encontrada.")
                
        # Lógica para os blocos BASAIS (Repouso)
        elif 'basal' in arquivo:
            if raw.times[-1] > 3.0: 
                raw.crop(tmin=1.0)
            epochs = mne.make_fixed_length_epochs(raw, duration=2.0, preload=True, verbose=False)
            epochs.save(caminho_saida, overwrite=True, verbose=False)
            print(f" {paciente_id}: Sucesso (Basal)")

    except Exception as e:
        print(f" Erro em {arquivo}: {e}")

print("\n Epocagem finalizada.")

Iniciando epocagem para 126 blocos...
 TEA_G_26_basal_pos: Sucesso (Basal)
 TEA_G_26_basal_pre: Sucesso (Basal)
 TEA_G_26_faces: Sucesso (30 épocas)
 TEA_G_28_basal_pos: Sucesso (Basal)
 TEA_G_28_basal_pre: Sucesso (Basal)
 TEA_G_28_faces: Sucesso (30 épocas)
 TEA_G_30_basal_pos: Sucesso (Basal)
 TEA_G_30_basal_pre: Sucesso (Basal)
 TEA_G_30_faces: Sucesso (30 épocas)
 TEA_G_40_basal_pos: Sucesso (Basal)
 TEA_G_40_basal_pre: Sucesso (Basal)
 TEA_G_40_faces: Sucesso (30 épocas)
 TEA_L_16_basal_pos: Sucesso (Basal)
 TEA_L_16_basal_pre: Sucesso (Basal)
 TEA_L_16_faces: Sucesso (30 épocas)
 TEA_L_17_basal_pos: Sucesso (Basal)
 TEA_L_17_basal_pre: Sucesso (Basal)
 TEA_L_17_faces: Sucesso (30 épocas)
 TEA_L_24_basal_pos: Sucesso (Basal)
 TEA_L_24_basal_pre: Sucesso (Basal)
 TEA_L_24_faces: Sucesso (30 épocas)
 TEA_L_25_basal_pos: Sucesso (Basal)
 TEA_L_25_basal_pre: Sucesso (Basal)
 TEA_L_25_faces: Sucesso (27 épocas)
 TEA_L_27_basal_pos: Sucesso (Basal)
 TEA_L_27_basal_pre: Sucesso (Basal)


In [15]:
# 02. EXTRAÇÃO DE FEATURES (PSD, ASSIMETRIA E ERP/N170)

print(" Iniciando Extração...")

arquivos_epocados = sorted([f for f in os.listdir(DIR_EPOCHS) if f.endswith('.fif')])
resultados = []

def calcular_assimetria_alfa(dados_psd):
    assimetrias = {}
    try:
        if 'Alpha_F4' in dados_psd and 'Alpha_F3' in dados_psd:
            assimetrias['Alpha_Asym_Frontal'] = np.log(dados_psd['Alpha_F4']) - np.log(dados_psd['Alpha_F3'])
        if 'Alpha_P4' in dados_psd and 'Alpha_P3' in dados_psd:
            assimetrias['Alpha_Asym_Parietal'] = np.log(dados_psd['Alpha_P4']) - np.log(dados_psd['Alpha_P3'])
    except:
        pass
    return assimetrias

# Mapeamento para agrupar as siglas diferentes no mesmo nome de condição
MAPA_CONDICOES = {
    'FF': 'Task/Face/Feliz',  'F': 'Task/Face/Feliz',
    'FN': 'Task/Face/Neutra', 'N': 'Task/Face/Neutra',
    'FR': 'Task/Face/Raiva',  'R': 'Task/Face/Raiva'
}

for arquivo in arquivos_epocados:
    caminho_epoch = os.path.join(DIR_EPOCHS, arquivo)
    nome_base = arquivo.replace('-epo.fif', '')
    
    partes_nome = nome_base.split('_')
    is_tea = 'TEA' in partes_nome[0].upper()
    grupo = 'TEA' if is_tea else 'Control'
    id_paciente = "_".join(partes_nome[:3]) if is_tea else "_".join(partes_nome[:2])
    
    try:
        epochs = mne.read_epochs(caminho_epoch, preload=True, verbose=False)
        
        # Fluxo 1: FACES
        if 'faces' in nome_base:
            # Itera sobre os eventos que realmente existem nesse arquivo
            for evento_presente in epochs.event_id.keys():
                if evento_presente in MAPA_CONDICOES:
                    nome_condicao = MAPA_CONDICOES[evento_presente]
                    epochs_condicao = epochs[evento_presente]
                    
                    linha = {'ID': id_paciente, 'Condicao': nome_condicao, 'Tipo': 'Task', 'Grupo': grupo}
                    
                    # 1. Extração PSD
                    psd = epochs_condicao.compute_psd(method='multitaper', fmin=4.0, fmax=40.0, verbose=False)
                    psd_data = psd.get_data().mean(axis=0)
                    freqs = psd.freqs
                    
                    dados_psd = {}
                    for ch_idx, canal in enumerate(epochs_condicao.ch_names):
                        if canal in CANAIS_PSD:
                            for banda, (fmin, fmax) in BANDAS_FREQ.items():
                                idx_banda = np.logical_and(freqs >= fmin, freqs <= fmax)
                                dados_psd[f"{banda}_{canal}"] = psd_data[ch_idx, idx_banda].mean()
                                
                    linha.update(dados_psd)
                    linha.update(calcular_assimetria_alfa(dados_psd))
                    
                    # 2. Extração ERP (N170)
                    evoked = epochs_condicao.average()
                    dados_erp = {}
                    for canal in CANAIS_N170:
                        if canal in evoked.ch_names:
                            canal_data = evoked.copy().pick([canal]).crop(tmin=JANELA_N170[0], tmax=JANELA_N170[1])
                            data_array = canal_data.get_data()[0] * 1e6
                            times_array = canal_data.times * 1000
                            
                            min_idx = np.argmin(data_array)
                            dados_erp[f"N170_Amp_{canal}"] = data_array[min_idx]
                            dados_erp[f"N170_Lat_{canal}"] = times_array[min_idx]
                            
                    linha.update(dados_erp)
                    resultados.append(linha)

        # Fluxo 2: Basal (repouso)
        else:
            nome_condicao = 'Resting/Pre' if 'basal_pre' in nome_base else 'Resting/Pos'
            linha = {'ID': id_paciente, 'Condicao': nome_condicao, 'Tipo': 'Resting', 'Grupo': grupo}
            
            psd = epochs.compute_psd(method='multitaper', fmin=4.0, fmax=40.0, verbose=False)
            psd_data = psd.get_data().mean(axis=0)
            freqs = psd.freqs
            
            dados_psd = {}
            for ch_idx, canal in enumerate(epochs.ch_names):
                if canal in CANAIS_PSD:
                    for banda, (fmin, fmax) in BANDAS_FREQ.items():
                        idx_banda = np.logical_and(freqs >= fmin, freqs <= fmax)
                        dados_psd[f"{banda}_{canal}"] = psd_data[ch_idx, idx_banda].mean()
                        
            linha.update(dados_psd)
            linha.update(calcular_assimetria_alfa(dados_psd))
            resultados.append(linha)
            
    except Exception as e:
        print(f" Erro ao extrair features de {arquivo}: {e}")

# Construção da Tabela Final
df_final = pd.DataFrame(resultados)
df_final = df_final.drop_duplicates(subset=['ID', 'Condicao'], keep='first')

caminho_csv = os.path.join(DIR_REPORTS, 'tabela_features_eeg_completa.csv')
df_final.to_csv(caminho_csv, index=False)

print("\n" + "="*50)
print(" EXTRAÇÃO DE CARACTERÍSTICAS CONCLUÍDA!")
print(f"Tabela de Machine Learning salva em: {caminho_csv}")
print(f"Total de linhas extraídas: {len(df_final)}")
qtd_feliz = len(df_final[df_final['Condicao'] == 'Task/Face/Feliz'])
print(f"Checagem de integridade -> Sujeitos em Face Feliz: {qtd_feliz}")
print("="*50)

 Iniciando Extração...



 EXTRAÇÃO DE CARACTERÍSTICAS CONCLUÍDA!
Tabela de Machine Learning salva em: ../reports/tabela_features_eeg_completa.csv
Total de linhas extraídas: 210
Checagem de integridade -> Sujeitos em Face Feliz: 42
